In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
df1 = pd.read_csv('fake_job_postings.csv')

In [4]:
df1.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [5]:
df1.shape

(17880, 18)

In [6]:
df=df1[['description','requirements','fraudulent']]
df.head()

,description,requirements,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0
1,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0
2,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0
3,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0


In [7]:
###Drop Nan Values
df=df.dropna()

In [8]:
df.shape

(15185, 3)

In [9]:
## Get the Independent Features

X=df.drop('fraudulent',axis=1)

In [10]:
## Get the Dependent features
y=df['fraudulent']

In [11]:
y.value_counts()

0    14473
1      712
Name: fraudulent, dtype: int64

In [12]:
import tensorflow as tf
tf.__version__

'2.9.1'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [14]:
voc_size=5000

>  ****One-hot Representation****

In [15]:
message = X.copy()

In [16]:
message['description'][1]

"Organised - Focused - Vibrant - Awesome!Do you have a passion for customer service? Slick typing skills? Maybe Account Management? ...And think administration is cooler than a polar bear on a jetski? Then we need to hear you!\xa0We are the Cloud Video Production Service and opperating on a glodal level. Yeah, it's pretty cool. Serious about\xa0delivering a world class product and excellent customer service.Our rapidly expanding business is looking for a talented Project Manager to manage the successful delivery of video projects, manage client communications and drive the production process. Work with some of the coolest brands on the planet and learn from a global team that are representing NZ is a huge way!We are entering the next growth stage of our business and growing quickly internationally. \xa0Therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 Seconds, the worlds Cloud Video Production Service -\xa0http://90

In [17]:
message.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asrai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['description'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
corpus[1]

'organis focus vibrant awesom passion custom servic slick type skill mayb account manag think administr cooler polar bear jetski need hear cloud video product servic opper glodal level yeah pretti cool seriou deliv world class product excel custom servic rapidli expand busi look talent project manag manag success deliveri video project manag client commun drive product process work coolest brand planet learn global team repres nz huge way enter next growth stage busi grow quickli intern therefor posit burst opportun right person enter busi right time second world cloud video product servic http url fbe afac cd c f b eef e e f ca dd second world cloud video product servic enabl brand agenc get high qualiti onlin video content shot produc anywher world fast afford manag seamlessli cloud purchas publish second remov hassl cost risk speed issu work regular video product compani manag everi aspect video project beauti onlin experi grow network rate video profession countri dedic product suc

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]

[3671,
 4398,
 4815,
 3771,
 3635,
 1288,
 4661,
 3603,
 296,
 1050,
 4768,
 1840,
 2040,
 3630,
 1111,
 4094,
 3346,
 4390,
 4060,
 4008,
 3384,
 850,
 2281,
 3624,
 4661,
 410,
 2393,
 1097,
 913,
 4312,
 2401,
 4994,
 1809,
 1142,
 1282,
 3624,
 3566,
 1288,
 4661,
 2954,
 1271,
 4494,
 3430,
 2551,
 2529,
 2040,
 2040,
 1942,
 3751,
 2281,
 2529,
 2040,
 2150,
 4460,
 2501,
 3624,
 3984,
 2210,
 1714,
 2723,
 3121,
 4215,
 4175,
 332,
 3237,
 4563,
 4333,
 3203,
 2968,
 3776,
 659,
 176,
 4494,
 2251,
 3919,
 3046,
 4187,
 2159,
 2573,
 2288,
 249,
 4370,
 2968,
 4494,
 249,
 1804,
 598,
 1142,
 850,
 2281,
 3624,
 4661,
 3317,
 2968,
 1271,
 3033,
 1040,
 4271,
 3948,
 2371,
 4345,
 4740,
 4740,
 3948,
 656,
 1107,
 598,
 1142,
 850,
 2281,
 3624,
 4661,
 3759,
 2723,
 4193,
 2439,
 1373,
 1551,
 3381,
 2281,
 342,
 2186,
 2071,
 1209,
 1142,
 2850,
 182,
 2040,
 922,
 850,
 3281,
 4571,
 598,
 1668,
 1690,
 2181,
 587,
 1832,
 2536,
 2210,
 1642,
 2281,
 3624,
 2312,
 2040,
 2362

****Embedding Representation****

In [23]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[3420  342 2040 ... 1326  569 4892]
 [1942   83 3786 ... 2195 1021  341]
 [   0    0    0 ... 4898  290 1374]
 ...
 [3622 3412 2440 ... 4594 1023 1613]
 [4005   44  549 ... 4682 3046 2529]
 [3385 4210  434 ... 3391 2439  269]]


In [24]:
embedded_docs[0]

array([3420,  342, 2040, 1718, 3101, 4113, 3482, 4243, 3925,  163,  402,
        402, 3925,  656, 1327,  744, 3925, 2567, 3925,  348, 1694, 3741,
       4936, 1815, 3375, 1111, 2210, 1824, 2521, 2880, 4211,  947, 1247,
       4960, 2145, 2159, 1435, 1326,  569, 4892])

In [25]:
## Creating model
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100))) ##Just add bidirectional!!, except it would just behave as normal LSTM Model
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 50)            250000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              120800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 371,001
Trainable params: 371,001
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs),y.shape

(15185, (15185,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final[1]

array([1942,   83, 3786, 2861, 4964, 4879, 3124,  598, 2071, 3971, 2281,
       2238, 4175, 2723, 4081, 1142, 4115, 4081,  999, 1143, 3837, 1941,
       1396, 3375, 3592, 4042, 3910,  358, 3741,  819, 3592, 3375,  666,
        249, 3509, 2825, 4690, 2195, 1021,  341])

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)

In [30]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64)

Epoch 1/12
178/178 [==============================] - 31s 128ms/step - loss: 0.2056 - accuracy: 0.9541 - val_loss: 0.1647 - val_accuracy: 0.9497
Epoch 2/12
178/178 [==============================] - 21s 120ms/step - loss: 0.0950 - accuracy: 0.9717 - val_loss: 0.1420 - val_accuracy: 0.9637
Epoch 3/12
178/178 [==============================] - 21s 119ms/step - loss: 0.0557 - accuracy: 0.9839 - val_loss: 0.1447 - val_accuracy: 0.9663
Epoch 4/12
178/178 [==============================] - 21s 116ms/step - loss: 0.0307 - accuracy: 0.9922 - val_loss: 0.1462 - val_accuracy: 0.9694
Epoch 5/12
178/178 [==============================] - 21s 115ms/step - loss: 0.0146 - accuracy: 0.9960 - val_loss: 0.1479 - val_accuracy: 0.9705
Epoch 6/12
178/178 [==============================] - 21s 117ms/step - loss: 0.0092 - accuracy: 0.9974 - val_loss: 0.1966 - val_accuracy: 0.9689
Epoch 7/12
178/178 [==============================] - 21s 116ms/step - loss: 0.0057 - accuracy: 0.9985 - val_loss: 0.1943 - val_ac

**Model Performance and Accuracy**

In [32]:
y_pred=(model1.predict(X_test)>0.5).astype('int32')
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

119/119 [==============================] - 4s 20ms/step


array([[3557,   28],
       [  95,  117]], dtype=int64)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9676060047405847

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      3585
           1       0.81      0.55      0.66       212

    accuracy                           0.97      3797
   macro avg       0.89      0.77      0.82      3797
weighted avg       0.96      0.97      0.96      3797



In [53]:
# input_text = ['Data Entry Admin/Clerical Positions - Work From Home ']
words=input('Data Entry Admin/Clerical Positions - Work From Home ')
onehot_repr=[one_hot(words,voc_size)] 
sent_length1=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length1)

# input_data = vect.transform(input_text)

prediction = model1.predict(embedded_docs)
prediction

if (prediction[0] == 1):
    print("This advertisement belonging to fake job post category")
else:
    print("This advertisement belonging to real job post category")  

Data Entry Admin/Clerical Positions - Work From Home Director of Strategy | San Mateo, CAÂ Interested in a company that has doubled annually for the last three years? Benchmarking in the top percentile of SaaS companies that have ultimately gone public? Deeply capitalized? Â Backed by investors such as Battery Ventures and Intel Capital?Â DescriptionThis role is responsible for building and leading the newly formed Corporate Strategy function. Â  It will report to a senior executive and will be responsible for identifying and answering a host of strategic questions, including growth, international expansion, acquisitions, and functional issues. Â  Â It will be an extremely visible role in the company and work across the entire organization. Â  This role will also be responsible for recruiting additional talent into the function, developing operational processes, and designing the talent management/career path for staff. Â  Â Due to the rapid growth of the company, this role will have o

In [54]:
model1.save('job_model.h5')